## Pandas analysis

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a couple of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [1]:
# If you didn't download it yet, please get the relevant file now!
!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ~/data/

--2024-11-28 08:57:15--  https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.69.18, 2620:100:6025:18::a27d:4512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.69.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/pkkpoxlm7beasryexpdf8/data_000637.txt?rlkey=rkm2em1v57hewglzelmin21c9 [following]
--2024-11-28 08:57:15--  https://www.dropbox.com/scl/fi/pkkpoxlm7beasryexpdf8/data_000637.txt?rlkey=rkm2em1v57hewglzelmin21c9
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce7725d1bcb33ce115a80272628.dl.dropboxusercontent.com/cd/0/inline/CfMUfrZNBt9ZvLgPvhYfJvUVc0LT7oY37aIPxTq--3CcX6il8qEyVBVBoBr1jfCsg8G_Fda7RFdk5c793W9WSvVIxLkCptgTJdsxA-m-GZNnJZLmYtrdcEhsbNjl9tDX0Q0/file# [following]
--2024-11-28 08:57:16--  https://uce7725d1bcb33ce115a80272628.dl.dropboxusercontent.com/cd/0/inline/CfMUfrZNBt

1\. Create a Pandas DataFrame reading N rows of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k.

2\. Find out the number of BX in a ORBIT (the value 'x').

3\. Find out how much the data taking lasted. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading out the whole dataset.

4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information).

5\. Replace the values (all 1) of the HEAD column randomly with 0 or 1.

6\. Create a new DataFrame that contains only the rows with HEAD=1.

7\. Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel

8\. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

9\. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139

In [84]:
import pandas as pd
import numpy as np

def read_N(N: int):
    data = pd.read_csv("~/data/data_000637.txt")
    if( N > len(data.index) or N < 10000):
        print("Choose a valid N. Setting N max")
        N = len(data.index)
    data = data[data.index < N]
    return data

data = read_N(2000)

Choose a valid N. Setting N max


In [85]:
#Counting how many BX_COUNTER are needed for ORBIT_CNT to be incremented by one
max_bx = np.max(data["BX_COUNTER"])
print("There are ", max_bx , " in 1 ORBIT_CNT")


There are  3563  in 1 ORBIT_CNT


In [86]:
%timeit read_N(30000)

272 ms ± 8.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [87]:
max_tdc_meas = np.max(data["TDC_MEAS"])
data["ABS_TIME"] = data["ORBIT_CNT"] + data["BX_COUNTER"] * max_bx + data["TDC_MEAS"] * max_tdc_meas
data.sort_values(by="ABS_TIME")
data.tail(10)

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,ABS_TIME
1310710,1,1,36,3869211171,427,13,3870732949
1310711,1,1,39,3869211171,430,0,3870743261
1310712,1,1,35,3869211171,429,4,3870739814
1310713,1,0,64,3869211171,758,18,3871912447
1310714,1,0,60,3869211171,762,2,3871926235
1310715,1,0,62,3869211171,762,14,3871926583
1310716,1,1,4,3869211171,763,11,3871930059
1310717,1,0,64,3869211171,764,0,3871933303
1310718,1,0,139,3869211171,769,0,3871951118
1310719,1,0,61,3869211171,762,18,3871926699


In [89]:
data["HEAD"] = np.random.randint(0,2,len(data)) 
data = data[data["HEAD"]==1]
data.head(10)

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,ABS_TIME
7,1,0,139,3869200167,2776,0,3879091055
10,1,1,7,3869200167,2785,4,3879123238
13,1,0,36,3869200167,2791,23,3879145167
16,1,1,8,3869200167,2787,14,3879130654
20,1,1,10,3869200167,2789,14,3879137780
21,1,0,57,3869200167,2789,10,3879137664
29,1,0,60,3869200167,3083,20,3880185476
32,1,0,61,3869200167,3085,25,3880192747
41,1,0,56,3869200167,3177,20,3880520398
50,1,0,62,3869200168,21,9,3869275252


In [95]:
import matplotlib.pyplot as plt

data_FPGA0 = data[data["FPGA"]==0]
data_FPGA1 = data[data["FPGA"]==1]

len(data_FPGA0["TDC_CHANNEL"]==5)


230763